In [3]:
#não executar, apenas mandar para o final para adicionar palavras

with open('palavras_termo.txt','w', encoding = 'utf8') as f:
    for palavra in palavras:
        f.write("%s\n" % palavra)
    print('done')


NameError: name 'palavras' is not defined

In [41]:
import pandas as pd
import numpy as np

In [45]:
import unicodedata
import re

def remove_acentos(string):
    normalized = unicodedata.normalize('NFD', string)
    return re.sub(r'[\u0300-\u036f]', '', normalized).casefold()

In [174]:
def conta_vogais(palavra):
    
    contador = 0
    vogais = 'aeiouAEIOU'
    allowed = frozenset('aeiouAEIOU')
    
    for letra in palavra:
        if letra in vogais:
            contador += 1
    vogais_unicas = len(allowed.intersection(palavra))
    
    return vogais_unicas, contador


conta_vogais('perde')    



(1, 2)

In [175]:
palavras = []
file = open('br-utf8.txt', 'r', encoding = 'utf8')

while True:
    line = file.readline()
    palavra = line.strip().replace('\n','')
    if not line:
        break
    if len(palavra) == 5:
        #print(palavra)
        palavras.append(remove_acentos(palavra))
        

file.close()


#nesse caso não tem a palavra lermo, então vamos remover da lista e rodar novamente
#criar uma função para quando isso acontecer e criar uma lista das palavras que o termo não aceita!
#assim podemos ir melhorando o algoritmo

palavras.remove('lermo')
palavras.remove('canaa')
palavras.remove('propo')
palavras.remove('coemo')
palavras.remove('doemo')
palavras.remove('jobim')
palavras.remove('sorta')
palavras.remove('suamo')


#print(len(palavras))
df = pd.DataFrame(palavras)
df.drop_duplicates(inplace = True)
df.rename(columns = {0:'palavra'}, inplace = True)

df['n_vogais'] = df['palavra'].apply(lambda x: conta_vogais(x)[1])
df['vogais_unicas'] = df['palavra'].apply(lambda x: conta_vogais(x)[0])


df['letra_0'] = df.palavra.str[0]
df['letra_1'] = df.palavra.str[1]
df['letra_2'] = df.palavra.str[2]
df['letra_3'] = df.palavra.str[3]
df['letra_4'] = df.palavra.str[4]
df.to_csv('br-utf8.csv', index = False)

df.head()

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
0,aarao,4,2,a,a,r,a,o
1,abner,2,2,a,b,n,e,r
2,acaia,4,2,a,c,a,i,a
3,acker,2,2,a,c,k,e,r
4,aires,3,3,a,i,r,e,s


In [128]:
df.to_csv('br-utf8.csv', index = False)

# Função de sortear palavra
estou usando esse metodo até criar um esquema de pontuação para as palavras

(depois necessário criação de CSV com a pontuação e todo a manipulação de dados, para que no final só fique o filtro!)

In [311]:
import random

def sorteia_palavra(lista_palavra):
    if len(lista_palavra) == 1:
        return lista_palavra[0]
    else:
        return lista_palavra[random.randint(0,(len(lista_palavra)-1))]



In [51]:
melhores_palavras_para_vogais = df[df.vogais_unicas == 4]
#escolher entre essas palavras para começar

# Startando o termoo no navegador
aqui seria onde comecei a entender como fazer a busca pela palavra e tentar criar alguma metrica pra achar letras nas strings


como proceder com palavras que tem letras repetidas?

aqui podemos fazer uma condição para cada posição estar com a letra certa


while pos1 == wrongletter or pos2 == wrongletter or pos3 == wrongletter or pos4 == wrongletter or pos5 == wrongletter

e ir lupando nas tentativas que temos, que aparentemente são 6

In [111]:
df.loc[df.palavra == 'nervo',:]

,palavra,n_vogais,vogais_unicas
3628,nervo,2,2


esse é o pensamento inicial, depois que começar o scraping no site e inserção poderemos saber quando acertarmos a posição da letra. 

Quando isso acontecer será necessário trocar essa função para adicionar só palavras com letra naquela posição!

teoricamente aqui conseguimos resolver mais ou menos o termo, agora é pesquisar novas metricas e testar nos outros dias, melhorando o filtro e a seleção, também seria legal rankear as palavras mais comuns e ver se conseguimos gerar um peso de palavras

- trabalhar em melhorar as funções e como elas vão conversar entre elas

-----------------------------

Palavras excluidas do dicionario baixado da usp:
- lermo
- canaa
- propo
- coemo
- doemo
- jobim
- suamo
- sorta

-----------------------------

# Organizando todas as funções!

- Carregar o DF
- listar as palavras melhroes rankeadas para começar o jogo
- iniciar o jogo
- escolher uma palavra da lista de mais rankeadas
- mandar a primeira palavra para o jogo
- retornar dicionario de resultado da primeira palavra

## Core Functions - Analitics

In [1]:
import pandas as pd

def carrega_dataframe_csv(caminho):
    return pd.read_csv(caminho)

In [2]:
import random

def sorteia_palavra(lista_palavra):
    if len(lista_palavra) == 1:
        return lista_palavra[0]
    else:
        return lista_palavra[random.randint(0,(len(lista_palavra)-1))]

In [3]:
#recebe lista de letra e posicao correta
#lista_exemplo = ['letra',posicao]

def letra_na_posicao(lista_letra_correta, df):
    
    if len(lista_letra_correta) == 0:
        return df
    else:
        for letra, posicao in lista_letra_correta:
            if posicao == 0:
                mask = df.letra_0 == letra
                df = df.loc[mask,:]
            elif posicao == 1:
                mask = df.letra_1 == letra
                df = df.loc[mask,:]
            elif posicao == 2:
                mask = df.letra_2 == letra
                df = df.loc[mask,:]
            elif posicao == 3:
                mask = df.letra_3 == letra
                df = df.loc[mask,:]
            elif posicao == 4:
                mask = df.letra_4 == letra
                df = df.loc[mask,:]

    return df


In [4]:
#lista_de_letras_com_posicao = [['g',2]] --- exemplo
#provavelmente será necessário criar uma função que descompacte o dicionario que retorna do site em uma lista com listas

def palavras_com_letra_posicao_errada(lista_de_letras_com_posicao, df):   
    lista = []
    for indice1,indice2 in lista_de_letras_com_posicao:
        lista.append(indice1)
        
        condicao = indice2
        if condicao == 0:
            mask = df.letra_1.isin(lista) | df.letra_2.isin(lista) | df.letra_3.isin(lista) | df.letra_4.isin(lista)
            df = df.loc[mask,:]
        elif condicao == 1:
            mask = df.letra_0.isin(lista) | df.letra_2.isin(lista) | df.letra_3.isin(lista) | df.letra_4.isin(lista) 
            df = df.loc[mask,:]
        elif condicao == 2:
            mask = df.letra_0.isin(lista) | df.letra_1.isin(lista) | df.letra_3.isin(lista) | df.letra_4.isin(lista)
            df = df.loc[mask,:]
        elif condicao == 3:
            mask = df.letra_0.isin(lista) | df.letra_1.isin(lista) | df.letra_2.isin(lista) | df.letra_4.isin(lista)
            df = df.loc[mask,:]
        elif condicao == 4:
            mask = df.letra_0.isin(lista) | df.letra_1.isin(lista) | df.letra_2.isin(lista) | df.letra_3.isin(lista)
            df = df.loc[mask,:]

    return df

In [5]:
def letras_nao_aceitas(letras, df):
    lista = []
    
    if len(letras) > 1:
        for i in letras:
            lista.append(i)
        mask = ~df.letra_0.isin(lista) & ~df.letra_1.isin(lista) & ~df.letra_2.isin(lista) & ~df.letra_3.isin(lista) & ~df.letra_4.isin(lista)
    else:
        mask = (df.letra_0 != letras) & (df.letra_1 != letras) & (df.letra_2 != letras) & (df.letra_3 != letras) & (df.letra_4 != letras)

    return df.loc[mask,:]



In [220]:
def letras_nao_aceitas(letras, df, letras_aceitas):
    lista = []


    for letra, indice,status_letras_aceitas in letra_n:
        if status_letras_aceitas == False:
            lista.append(letra)
        else:
            mask = df[f'letra_{indice}'] != letra
            df = df.loc[mask,:]

        if len(lista) > 0:
            mask = ~df.letra_0.isin(lista) & ~df.letra_1.isin(lista) & ~df.letra_2.isin(lista) & ~df.letra_3.isin(lista) & ~df.letra_4.isin(lista)    
            df = df.loc[mask,:]
        
    return df


In [197]:
def letras_nao_aceitas(letras, df):
    for letra, indice in letras:
        mask = df[f'letra_{indice}'] != letra
        df = df.loc[mask,:]

    return df



In [215]:
caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)
df.head(5)

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
0,aarao,4,2,a,a,r,a,o
1,abner,2,2,a,b,n,e,r
2,acaia,4,2,a,c,a,i,a
3,acker,2,2,a,c,k,e,r
4,aires,3,3,a,i,r,e,s


In [216]:
letra_n = [('a',0,True),('e',3,False)]
letras_aceitas = 'a'
lista = []


for letra, indice,status_letras_aceitas in letra_n:
    print(letra)
    print(indice)
    print(status_letras_aceitas)
    if status_letras_aceitas == False:
        lista.append(letra)
    else:
        mask = df[f'letra_{indice}'] != letra
        df = df.loc[mask,:]

    if len(lista) > 0:
        mask = ~df.letra_0.isin(lista) & ~df.letra_1.isin(lista) & ~df.letra_2.isin(lista) & ~df.letra_3.isin(lista) & ~df.letra_4.isin(lista)    
        df = df.loc[mask,:]
        
df
    

a
0
True
e
3
False


,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
17,bagda,2,1,b,a,g,d,a
18,bangu,2,2,b,a,n,g,u
24,borba,2,2,b,o,r,b,a
25,bruno,2,2,b,r,u,n,o
26,carla,2,1,c,a,r,l,a
...,...,...,...,...,...,...,...,...
5412,umida,3,3,u,m,i,d,a
5413,umido,3,3,u,m,i,d,o
5414,unica,3,3,u,n,i,c,a
5415,unico,3,3,u,n,i,c,o


In [217]:
df[df.letra_1 == 'a']

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
17,bagda,2,1,b,a,g,d,a
18,bangu,2,2,b,a,n,g,u
26,carla,2,1,c,a,r,l,a
33,catia,3,2,c,a,t,i,a
36,darci,2,2,d,a,r,c,i
...,...,...,...,...,...,...,...,...
5287,zanza,2,1,z,a,n,z,a
5289,zanzo,2,2,z,a,n,z,o
5290,zarpa,2,1,z,a,r,p,a
5292,zarpo,2,2,z,a,r,p,o


In [218]:
df[df.palavra == 'pavor']

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
3535,pavor,2,2,p,a,v,o,r


## Scraping functions

In [6]:
import pandas as pd
import numpy as np
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
#com a mudança pro selenium 4.0, algumas coisas mudaram ao inicializar, por exemplo o uso do Service


from bs4 import BeautifulSoup
from tqdm import tqdm

In [7]:
#inicializand o site
def inicializa_termo():
    service = Service(executable_path = './edg_drv/msedgedriver.exe') #mudança do selenium 4.0
    driver = webdriver.Edge(service=service)

    urlpage = 'https://term.ooo/'
    response = driver.get(urlpage)
    driver.implicitly_wait(5)

    x_path = '/html/body/wc-modal'
    ajuda_sempre_aberta = driver.find_element('xpath',x_path)
    ajuda_sempre_aberta.click()
    driver.implicitly_wait(5)
    return driver
    
    


In [97]:
#envia palavra
from time import sleep

def envia_palavra(palavra_sorteada):
    x_path_board = '/html/body'
    board_inicial = driver.find_element('xpath',x_path_board)
    for letra in palavra_sorteada:
        board_inicial.send_keys(letra)
        #board_inicial.implicitly_wait(3)#testar, se não funcionar voltar para -> driver.implicitly_wait(0.5)
        sleep(1)

    
    board_inicial.send_keys(Keys.ENTER)
    #board_inicial.implicitly_wait(5)
    sleep(3)
    #print('palavra-enviada')#apenas pra finalidades de teste, retirar depois
    
def apaga_palavra():
    x_path_board = '/html/body'
    board_inicial = driver.find_element('xpath',x_path_board)

    for i in range(5):
        board_inicial.send_keys(Keys.BACKSPACE)
        sleep(0.5)
    

In [199]:
#retorna dicionario da tentiva de palavra enviada
# elemento_do_board = '[id="board0"]'

# shadow = driver.find_element(By.CSS_SELECTOR, elemento_do_board).shadow_root

#ainda pensando se vale a pena colocar como variavel global mesmo ou deixar dentro da função

def retorna_dicionario_respostas(variavel): #variavel = numero da tentativa

    elemento_do_board = '[id="board0"]'

    shadow = driver.find_element(By.CSS_SELECTOR, elemento_do_board).shadow_root
    letra = ''
    resposta = ''
    
    
    lista_da_palavra = [] #inicializa a lista
    elemento_primeira_linha_interno_para_shadow = f'[aria-label="palavra {variavel}"]'
    inner_shadow = shadow.find_element(By.CSS_SELECTOR, elemento_primeira_linha_interno_para_shadow).shadow_root


    for variavel in range(5):
        elemento_primeira_letra = f'[termo-pos="{str(variavel)}"]'
        elemento_html = inner_shadow.find_element(By.CSS_SELECTOR, elemento_primeira_letra)
        resposta = elemento_html.get_attribute('class')
        letra = remove_acentos(elemento_html.text.lower())
        if resposta != 'letter empty':
            lista_da_palavra.append((resposta, letra))


    return dict(zip(range(len(lista_da_palavra)), lista_da_palavra))

In [ ]:
#celulas excluidas sao uma tentativa de juntar a funçao de gerar o dicionario e descompactar

In [108]:
def descompacta_respostas(variavel, letras_aceitas):

    elemento_do_board = '[id="board0"]'

    shadow = driver.find_element(By.CSS_SELECTOR, elemento_do_board).shadow_root
    letra = ''
    resposta = ''
    
    
    lista_da_palavra = [] #inicializa a lista
    elemento_primeira_linha_interno_para_shadow = f'[aria-label="palavra {variavel}"]'
    inner_shadow = shadow.find_element(By.CSS_SELECTOR, elemento_primeira_linha_interno_para_shadow).shadow_root


    for variavel in range(5):
        elemento_primeira_letra = f'[termo-pos="{str(variavel)}"]'
        elemento_html = inner_shadow.find_element(By.CSS_SELECTOR, elemento_primeira_letra)
        resposta = elemento_html.get_attribute('class')
        letra = remove_acentos(elemento_html.text.lower())
        if resposta != 'letter empty':
            lista_da_palavra.append([resposta, letra])


    dicionario = dict(zip(range(len(lista_da_palavra)), lista_da_palavra))

    letras_erradas = ''
    letras_lugar_errado = []
    letras_corretas = []

    for item in dicionario:
        letra = remove_acentos(dicionario[item][1]).lower()
        condicao = letra in letras_aceitas
        if dicionario[item][0] == 'letter wrong' and not(condicao):
            letras_erradas += str(dicionario[item][1])
        elif dicionario[item][0] == 'letter place':
            letras_aceitas += letra
            letras_lugar_errado.append([dicionario[item][1], item])
        elif dicionario[item][0] == 'letter right':
            letras_aceitas += letra
            letras_corretas.append([dicionario[item][1], item])
            
            
    return letras_erradas, letras_lugar_errado, letras_corretas, letras_aceitas

In [31]:
variavel = 1
elemento_do_board = '[id="board0"]'

shadow = driver.find_element(By.CSS_SELECTOR, elemento_do_board).shadow_root


lista_da_palavra = [] #inicializa a lista
elemento_primeira_linha_interno_para_shadow = f'[aria-label="palavra {variavel}"]'
inner_shadow = shadow.find_element(By.CSS_SELECTOR, elemento_primeira_linha_interno_para_shadow).shadow_root


for variavel in range(5):
    elemento_primeira_letra = f'[termo-pos="{str(variavel)}"]'
    elemento_html = inner_shadow.find_element(By.CSS_SELECTOR, elemento_primeira_letra)
    resposta = elemento_html.get_attribute('class')
    letra = elemento_html.text.lower()
    if resposta != 'letter empty':
        lista_da_palavra.append([resposta, letra])


resultado_teste = dict(zip(range(len(lista_da_palavra)), lista_da_palavra))
resultado_teste

{0: ['letter wrong', 't'],
 1: ['letter wrong', 'ó'],
 2: ['letter wrong', 'r'],
 3: ['letter place', 'i'],
 4: ['letter right', 'o']}

In [109]:
def descompacta_dicionario(dicionario, letras_aceitas):

    letras_erradas = ''
    letras_lugar_errado = []
    letras_corretas = []

    for item in dicionario:
        letra = remove_acentos(dicionario[item][1]).lower()
        condicao = letra in letras_aceitas
        if dicionario[item][0] == 'letter wrong' and not(condicao):
            letras_erradas += str(dicionario[item][1])
        elif dicionario[item][0] == 'letter place':
            letras_aceitas += letra
            letras_lugar_errado.append([dicionario[item][1], item])
        elif dicionario[item][0] == 'letter right':
            letras_aceitas += letra
            letras_corretas.append([dicionario[item][1], item])
            
    return letras_erradas, letras_lugar_errado, letras_corretas, letras_aceitas

aqui foi necessário fazer varios for, pra primeiro:
- adicionar as letras aceitas e corretas nos lugares
- adicionar as letras em lugares errados
- adicionar letras erradas

isso se deu ao motivo de muitas vezes terem letras repetidas e o site reconhecer uma letra aceita como não aceita!


### isso pode parar de ocorrer se eu parar de lidar com as letras erradas e trabalhar do mesmo jeito das outras!

In [203]:
#aqui foi necessário fazer varios for, pra primeiro

def descompacta_dicionario(dicionario, letras_aceitas):

    letras_erradas = []
    letras_lugar_errado = []
    letras_corretas = []

    #primeiro sempre adicionando as letras corretas!
    for item in dicionario:
        letra = remove_acentos(dicionario[item][1]).lower()
        condicao = letra in letras_aceitas
        if dicionario[item][0] == 'letter right':
            letras_aceitas += letra
            letras_corretas.append((dicionario[item][1], item))

    #adicionando as letras corretas mas em posição errada!
    for item in dicionario:
        letra = remove_acentos(dicionario[item][1]).lower()
        condicao = letra in letras_aceitas        
        if dicionario[item][0] == 'letter place':
            letras_aceitas += letra
            letras_lugar_errado.append((dicionario[item][1], item))
            
    #e só por fim adicionar as letras erradas a equação    
    for item in dicionario:
        letra = remove_acentos(dicionario[item][1]).lower()
        condicao = letra in letras_aceitas
        if dicionario[item][0] == 'letter wrong':
            if dicionario[item][1] in letras_aceitas:
                letras_erradas.append((dicionario[item][1], item, True))
                #retornando true se a letra errada está na lista de letras aceitas
            else: 
                letras_erradas.append((dicionario[item][1], item, False))
                #retornando false se a letra errada está na lista de letras aceitas
    #será necessário criar uma condição só pra letra não aceita + letra aceita por causa do erro do site!
            
            
            
    return letras_erradas, letras_lugar_errado, letras_corretas, letras_aceitas

In [106]:
letra = 'o'
letras_aceitas = 'onz'

not(letra in letras_aceitas)

False

In [21]:
#função que vai dizer se ta funcionando ou não
def get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa):
#def get_notificacao(palavra_sorteada, df):
    vencedor = 'wc-notify'
    notificacao = driver.find_element(By.CSS_SELECTOR, vencedor)
    notificacao = notificacao.text
    #print(notificacao)
    terminou = True
    if notificacao == '':
        notificacao = 'palavra aceita'
        terminou = True
        
        
    elif notificacao == 'essa palavra não é aceita':
        #file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
        #nesse ponto precisei meio que criar ela recursivamente, pra ir filtrando as palavras que não poderiamos colocar!
        while terminou == True:
            file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
            #aqui vai ter uma mini função, que talvez eu encapsule em outro lugar
            df = df[df.palavra != palavra_sorteada]
            palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())

            x_path_board = '/html/body'
            board_inicial = driver.find_element('xpath',x_path_board)

            for i in range(5):
                board_inicial.send_keys(Keys.BACKSPACE)
                sleep(1)

            envia_palavra(palavra_sorteada)
            dicionario = retorna_dicionario_respostas(tentativa)
            letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

            df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
            df = letra_na_posicao(letras_corretas, df)
            df = letras_nao_aceitas(letras_erradas, df)
            
            terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id,dicionario,tentativa)
            
            
        
        
    else:
        terminou = False
        
    return terminou, df, notificacao, palavra_sorteada

FAZENDO UMA SEGUNDA VERSÃO DA FUNCAO ACIMA

In [187]:
#função que vai dizer se ta funcionando ou não
def get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas):
#def get_notificacao(palavra_sorteada, df):
    vencedor = 'wc-notify'
    notificacao = driver.find_element(By.CSS_SELECTOR, vencedor)
    notificacao = notificacao.text
    #print(notificacao)
    terminou = True
    if notificacao == '':
        notificacao = 'palavra aceita' #para input do resultados.txt
        terminou = True
        
        
    elif notificacao == 'essa palavra não é aceita':
        terminou = False
        #file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
        #nesse ponto precisei meio que criar ela recursivamente, pra ir filtrando as palavras que não poderiamos colocar!
        while terminou == False:
            
            
            #apagando palavra
            apaga_palavra()
            
            
            file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
            #aqui vai ter uma mini função, que talvez eu encapsule em outro lugar
            df = df[df.palavra != palavra_sorteada].reset_index(drop=True)
            palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
            
            #palavra_sorteada = sorteia_palavra(df.palavra)

            envia_palavra(palavra_sorteada)

            #pegar informações da palavra enviada
            dicionario = retorna_dicionario_respostas(tentativa)
            df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
#             print('-------- dentro do get_noficacao ------')
#             print('palavra_sorteada: ',palavra_sorteada)
#             print()
#             print('dicionario: ', dicionario)
#             print()
#             print('letras_aceitas: ',letras_aceitas)
#             print('tentativa: ', tentativa)
#             print('-------- fim do get_noficacao ------')
#             df.to_csv('debugando.csv')
            #df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
            #df.to_csv('debugando.csv')
            
            #terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id,dicionario,tentativa, letras_aceitas)
            terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
            #tentativa += 1 # foi o jeito de corrigir o erro, não sei como só isso resolveu, verificar deep dps
            
            
        
        
    else:
        terminou = False
        
    return terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa

In [189]:
def first_attempt(df):
    melhores_palavras_para_vogais = df[df.vogais_unicas == 4]
    #melhores_palavras_para_vogais = df[df.palavra == 'aureo']
    #return sorteia_palavra(melhores_palavras_para_vogais.palavra.values.tolist())
    return 'aureo'

In [162]:
def filtra_df(dicionario, df, letras_aceitas):
#         envia_palavra(palavra_sorteada)

#         #pegar informações da palavra enviada
#         dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas, letras_aceitas = descompacta_dicionario(dicionario, letras_aceitas)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        
        return df, letras_aceitas

In [166]:
def filtra_df_2(dicionario, df, letras_aceitas):
#         envia_palavra(palavra_sorteada)

#         #pegar informações da palavra enviada
#         dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas, letras_aceitas = descompacta_dicionario(dicionario, letras_aceitas)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        
        return df, letras_aceitas, letras_erradas, letras_lugar_errado, letras_corretas

# teste de inicialização e resolução

In [152]:
caminho = 'br-utf8.csv'
df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

while terminou == True:
    driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

    terminou, df, notificacao = get_notificacao(palavra_sorteada, df)
    print(f'tentativa: {tentativa} - palavra: {palavra_sorteada} - resultado: {notificacao}')
    tentativa += 1

ureia
tentativa: 0 - palavra: ureia - resultado: 
tentativa: 1 - palavra: zonzo - resultado: 
tentativa: 2 - palavra: tolho - resultado: 
tentativa: 3 - palavra: pombo - resultado: Fenomenal


In [155]:
caminho = 'br-utf8.csv'
df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

    terminou, df, notificacao = get_notificacao(palavra_sorteada, df)
    print(f'tentativa: {tentativa} - palavra: {palavra_sorteada} - resultado: {notificacao}')
    tentativa += 1

tentativa: 0 - palavra: vaiou - resultado: palavra aceita
tentativa: 1 - palavra: bolos - resultado: palavra aceita
tentativa: 2 - palavra: rombo - resultado: palavra aceita
tentativa: 3 - palavra: pombo - resultado: Fenomenal


In [14]:
from datetime import datetime

file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

            
    terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id}\n')
    tentativa += 1

file.close()

TypeError: get_notificacao() missing 1 required positional argument: 'dicionario'

# BLOCO DE CODIGO FINAL QUE EXECUTA E ESCREVE O QUE FOI FEITO

In [201]:
file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
#variavel_randomica = random.randint(0,1000)
variavel_randomica = 416
a = random.seed(variavel_randomica)
print(variavel_randomica)


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = descompacta_dicionario(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

            
    terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
    tentativa += 1

file.close()

416


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
dicionario_teste = {0: ['letter wrong', 'm'], 1: ['letter wrong', 'o'], 2: ['letter wrong', 'g'], 3: ['letter place', 'n'], 4: ['letter right', 'o']}

df = filtra_df(dicionario_teste, df)

In [116]:
letras_aceitas = ''

file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = 416
a = random.seed(variavel_randomica)
print(variavel_randomica)


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)
        
        #print(palavra_sorteada)
        df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
        
        #driver.implicitly_wait(10)
        sleep(1)
        #terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)
        
        #print(palavra_sorteada)
        df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
        #driver.implicitly_wait(30)
        sleep(1)
        #terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    print('tentativa: ',tentativa)
    print('palavra sorteada: ',palavra_sorteada) 
    print('letras aceitas: ',letras_aceitas)
    terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
    tentativa += 1
    if tentativa > 6:
        terminou = False

print('-------- fim da tentativa ------')        
print('Fim')
print('')
file.close()

416
tentativa:  0
palavra sorteada:  apeou
letras aceitas:  o
tentativa:  1
palavra sorteada:  jobim
letras aceitas:  o
-------- dentro do get_noficacao ------
palavra_sorteada:  torio

dicionario:  {0: ['letter wrong', 't'], 1: ['letter wrong', 'o'], 2: ['letter wrong', 'r'], 3: ['letter place', 'i'], 4: ['letter right', 'o']}

letras_aceitas:  oio
tentativa:  1
-------- fim do get_noficacao ------
tentativa:  2
palavra sorteada:  linho
letras aceitas:  oioino
tentativa:  3
palavra sorteada:  vindo
letras aceitas:  oioinoino
tentativa:  4
palavra sorteada:  finjo
letras aceitas:  oioinoinoino
tentativa:  5
palavra sorteada:  zinco
letras aceitas:  oioinoinoino


# NOVO BLOCO FINAL

In [178]:
letras_aceitas = ''

file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = random.randint(0,1000)
variavel_randomica = 259
a = random.seed(variavel_randomica)
print(variavel_randomica)




while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)
        
        #print(palavra_sorteada)
        df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
        
        #driver.implicitly_wait(10)
        sleep(1)

        #terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    #elif tentativa > 0 and tentativa < 7:
    elif tentativa > 0:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)
        print(dicionario)
        
        #print(palavra_sorteada)
        df.to_csv('debugando.csv')
        df, letras_aceitas, letras_erradas, letras_lugar_errado, letras_corretas = filtra_df_2(dicionario, df, letras_aceitas)
        #driver.implicitly_wait(30)
        sleep(1)

        #terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
#     print(dicionario)
    
#     print('tentativa: ',tentativa)
#     print('letras erradas: ', letras_erradas)
#     print('letras lugar errado: ', letras_lugar_errado)
#     print('letras corretas: ', letras_corretas)
#     print('palavra sorteada: ',palavra_sorteada) 
#     print('letras aceitas: ',letras_aceitas)
    terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
    tentativa += 1
    
    if tentativa > 6:
        terminou = False

file.close()

259
{0: ['letter wrong', 'u'], 1: ['letter wrong', 'n'], 2: ['letter wrong', 'i'], 3: ['letter place', 'a'], 4: ['letter place', 'o']}
tentativa:  0
letras erradas:  rm
letras lugar errado:  [['a', 0]]
letras corretas:  [['r', 4]]
palavra sorteada:  uniao
letras aceitas:  ao
{0: ['letter place', 'a'], 1: ['letter wrong', 'r'], 2: ['letter wrong', 'm'], 3: ['letter wrong', 'a'], 4: ['letter right', 'r']}
{0: ['letter place', 'a'], 1: ['letter wrong', 'r'], 2: ['letter wrong', 'm'], 3: ['letter wrong', 'a'], 4: ['letter right', 'r']}
tentativa:  1
letras erradas:  m
letras lugar errado:  [['a', 0]]
letras corretas:  [['r', 4]]
palavra sorteada:  armar
letras aceitas:  aora
{0: ['letter wrong', 'j'], 1: ['letter right', 'a'], 2: ['letter wrong', 'z'], 3: ['letter wrong', 'e'], 4: ['letter right', 'r']}
{0: ['letter wrong', 'j'], 1: ['letter right', 'a'], 2: ['letter wrong', 'z'], 3: ['letter wrong', 'e'], 4: ['letter right', 'r']}
tentativa:  2
letras erradas:  jze
letras lugar errado:  [

In [186]:
letras_aceitas = ''

file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = random.randint(0,1000)
variavel_randomica = 259
a = random.seed(variavel_randomica)
print(variavel_randomica)

#print('first')
#pegando a primeira palavra
palavra_sorteada = first_attempt(df)

envia_palavra(palavra_sorteada)

#pegar informações da palavra enviada
dicionario = retorna_dicionario_respostas(tentativa)

#print(palavra_sorteada)
#df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)

df, letras_aceitas, letras_erradas, letras_lugar_errado, letras_corretas = filtra_df_2(dicionario, df, letras_aceitas)

#driver.implicitly_wait(10)
sleep(1)
print(dicionario)
print('tentativa: ',tentativa)
print('letras erradas: ', letras_erradas)
print('letras lugar errado: ', letras_lugar_errado)
print('letras corretas: ', letras_corretas)
print('palavra sorteada: ',palavra_sorteada) 
print('letras aceitas: ',letras_aceitas)

tentativa += 1

while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)

    palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
    #print(palavra_sorteada)

    envia_palavra(palavra_sorteada)

    #pegar informações da palavra enviada
    dicionario = retorna_dicionario_respostas(tentativa)

    #print(palavra_sorteada)
    #df.to_csv('debugando.csv')
    df, letras_aceitas, letras_erradas, letras_lugar_errado, letras_corretas = filtra_df_2(dicionario, df, letras_aceitas)
    #driver.implicitly_wait(30)
    sleep(1)

        #terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    print(dicionario)
    
    print('tentativa: ',tentativa)
    print('letras erradas: ', letras_erradas)
    print('letras lugar errado: ', letras_lugar_errado)
    print('letras corretas: ', letras_corretas)
    print('palavra sorteada: ',palavra_sorteada) 
    print('letras aceitas: ',letras_aceitas)
    terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
    tentativa += 1
    
    if tentativa > 6:
        terminou = False

file.close()

259
{0: ['letter wrong', 'u'], 1: ['letter wrong', 'n'], 2: ['letter wrong', 'i'], 3: ['letter place', 'a'], 4: ['letter place', 'o']}
tentativa:  0
letras erradas:  uni
letras lugar errado:  [['a', 3], ['o', 4]]
letras corretas:  []
palavra sorteada:  uniao
letras aceitas:  ao
{0: ['letter place', 'v'], 1: ['letter right', 'a'], 2: ['letter wrong', 'i'], 3: ['letter right', 'o'], 4: ['letter wrong', 'u']}
tentativa:  1
letras erradas:  iu
letras lugar errado:  [['v', 0]]
letras corretas:  [['a', 1], ['o', 3]]
palavra sorteada:  vaiou
letras aceitas:  aoaov
{0: ['letter right done', 'p'], 1: ['letter right done', 'a'], 2: ['letter right done', 'v'], 3: ['letter right done', 'o'], 4: ['letter right done', 'r']}
tentativa:  2
letras erradas:  
letras lugar errado:  []
letras corretas:  []
palavra sorteada:  pavor
letras aceitas:  aoaov


259 variavel_randomica que estava dando erro ontem

In [205]:
letras_aceitas = ''

file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = random.randint(0,1000)
variavel_randomica = 92
a = random.seed(variavel_randomica)
print(variavel_randomica)

#print('first')
#pegando a primeira palavra
palavra_sorteada = first_attempt(df)

envia_palavra(palavra_sorteada)

#pegar informações da palavra enviada
dicionario = retorna_dicionario_respostas(tentativa)

#print(palavra_sorteada)
#df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)

df, letras_aceitas, letras_erradas, letras_lugar_errado, letras_corretas = filtra_df_2(dicionario, df, letras_aceitas)

#driver.implicitly_wait(10)
sleep(1)
print('------------------------------------')
print('palavra sorteada: ',palavra_sorteada) 
print(dicionario)
print('tentativa: ',tentativa)
print('letras erradas: ', letras_erradas)
print('letras lugar errado: ', letras_lugar_errado)
print('letras corretas: ', letras_corretas)
print('letras aceitas: ',letras_aceitas)

tentativa += 1

while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)

    palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
    #print(palavra_sorteada)

    envia_palavra(palavra_sorteada)

    #pegar informações da palavra enviada
    dicionario = retorna_dicionario_respostas(tentativa)

    #print(palavra_sorteada)
    #df.to_csv('debugando.csv')
    df, letras_aceitas, letras_erradas, letras_lugar_errado, letras_corretas = filtra_df_2(dicionario, df, letras_aceitas)
    #driver.implicitly_wait(30)
    sleep(1)

        #terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    print('------------------------------------')
    print('palavra sorteada: ',palavra_sorteada) 
    print(dicionario)
    
    print('tentativa: ',tentativa)
    print('letras erradas: ', letras_erradas)
    print('letras lugar errado: ', letras_lugar_errado)
    print('letras corretas: ', letras_corretas)
    
    print('letras aceitas: ',letras_aceitas)
    terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
    tentativa += 1
    
    if tentativa > 6:
        terminou = False

file.close()

92
------------------------------------
palavra sorteada:  aureo
{0: ('letter place', 'a'), 1: ('letter wrong', 'u'), 2: ('letter place', 'r'), 3: ('letter wrong', 'e'), 4: ('letter place', 'o')}
tentativa:  0
letras erradas:  [('u', 1), ('e', 3)]
letras lugar errado:  [('a', 0), ('r', 2), ('o', 4)]
letras corretas:  []
letras aceitas:  aro
------------------------------------
palavra sorteada:  rejas
{0: ('letter place', 'r'), 1: ('letter wrong', 'e'), 2: ('letter wrong', 'j'), 3: ('letter place', 'a'), 4: ('letter wrong', 's')}
tentativa:  1
letras erradas:  [('e', 1), ('j', 2), ('s', 4)]
letras lugar errado:  [('r', 0), ('a', 3)]
letras corretas:  []
letras aceitas:  arora
------------------------------------
palavra sorteada:  tarda
{0: ('letter wrong', 't'), 1: ('letter right', 'a'), 2: ('letter place', 'r'), 3: ('letter wrong', 'd'), 4: ('letter wrong', 'a')}
tentativa:  2
letras erradas:  [('t', 0), ('d', 3), ('a', 4)]
letras lugar errado:  [('r', 2)]
letras corretas:  [('a', 1)

In [209]:
file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = 52 #testando o q deu errado!
a = random.seed(variavel_randomica)
print(variavel_randomica)


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

            
    terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica}\n')
    tentativa += 1

file.close()

52


In [200]:

variavel_randomica = random.randint(0,1000)
print(variavel_randomica)
a = random.seed(variavel_randomica)
random.randint(0,1000)

664


804

In [199]:
a = random.seed(821)
random.randint(0,1000)

583

In [201]:
a = random.seed(664)
random.randint(0,1000)

804

In [150]:
#depois procurar pq nao ta funcionando essa funçaõ

#fazer função do bloco final

In [135]:
import nltk
local = 'livros.txt'
def abrir_arquivo(local):
    with open(local, 'r', encoding = 'utf8') as f:
        palavras = f.read()
    f.close()
    lista_tokens = nltk.tokenize.word_tokenize(palavras)
    lista_palavras = []
    for i in lista_tokens:
        if i.isalpha():
            if len(i) == 5:
                lista_palavras.append(remove_acentos(i.lower()))
            
    f.close()
    return lista_palavras

lista_palavras = abrir_arquivo(local)
lista_palavras

['fonte',
 'frase',
 'viver',
 'quero',
 'frase',
 'forma',
 'casar',
 'viver',
 'criar',
 'conto',
 'gozar',
 'minha',
 'penso',
 'quero',
 'ainda',
 'tenha',
 'corpo',
 'minha',
 'lenha',
 'desse',
 'quero',
 'ainda',
 'tenha',
 'minha',
 'assim',
 'penso',
 'ponho',
 'forma',
 'tomou',
 'nossa',
 'latim',
 'frase',
 'penso',
 'penso',
 'estou',
 'estar',
 'areia',
 'vento',
 'norte',
 'fundo',
 'achas',
 'tanta',
 'coval',
 'vazio',
 'praia',
 'raiva',
 'louco',
 'lugar',
 'praia',
 'gente',
 'nisto',
 'vacuo',
 'visto',
 'magoa',
 'vento',
 'norte',
 'pulso',
 'mundo',
 'levas',
 'areia',
 'tenho',
 'levar',
 'fosse',
 'fosse',
 'longe',
 'ideia',
 'noite',
 'chuva',
 'vento',
 'torvo',
 'vento',
 'lugar',
 'noite',
 'fazes',
 'crias',
 'mundo',
 'areia',
 'minha',
 'tedio',
 'lares',
 'razao',
 'atira',
 'paria',
 'vento',
 'forma',
 'noite',
 'termo',
 'almas',
 'tenho',
 'almas',
 'tenho',
 'mudei',
 'nunca',
 'tanto',
 'tenho',
 'calma',
 'sente',
 'sonho',
 'nasce',
 'minha',


In [136]:
def probabilidade(palavra):
    return frequencia[palavra]/total_de_palavras

frequencia = nltk.FreqDist(lista_palavras)
total_de_palavras = len(lista_palavras)
frequencia.most_common(10)

[('minha', 361),
 ('assim', 194),
 ('olhos', 186),
 ('muito', 163),
 ('agora', 149),
 ('ainda', 147),
 ('outra', 145),
 ('tinha', 137),
 ('mesmo', 129),
 ('disse', 127)]

In [121]:
len(lista_palavras)

9026

In [137]:
caminho = 'br-utf8.csv'
df = carrega_dataframe_csv(caminho)
df

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
0,aarao,4,2,a,a,r,a,o
1,abner,2,2,a,b,n,e,r
2,acaia,4,2,a,c,a,i,a
3,acker,2,2,a,c,k,e,r
4,aires,3,3,a,i,r,e,s
...,...,...,...,...,...,...,...,...
5419,unica,3,3,u,n,i,c,a
5420,unico,3,3,u,n,i,c,o
5421,urico,3,3,u,r,i,c,o
5422,uteis,3,3,u,t,e,i,s


In [138]:
df['probabilidade'] = df.palavra.apply(lambda x: probabilidade(x))
df.head()

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4,probabilidade
0,aarao,4,2,a,a,r,a,o,0.0
1,abner,2,2,a,b,n,e,r,0.0
2,acaia,4,2,a,c,a,i,a,0.0
3,acker,2,2,a,c,k,e,r,0.0
4,aires,3,3,a,i,r,e,s,0.0


In [139]:
df.sort_values(by=['probabilidade'], ascending = False)

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4,probabilidade
3051,minha,2,2,m,i,n,h,a,0.034329
550,assim,2,2,a,s,s,i,m,0.018448
3364,olhos,2,1,o,l,h,o,s,0.017687
3173,muito,3,3,m,u,i,t,o,0.015500
285,agora,3,2,a,g,o,r,a,0.014169
...,...,...,...,...,...,...,...,...,...
2025,fitem,2,2,f,i,t,e,m,0.000000
2024,fitei,3,2,f,i,t,e,i,0.000000
2020,fitai,3,2,f,i,t,a,i,0.000000
2019,fisgo,2,2,f,i,s,g,o,0.000000


In [140]:
df[df.palavra == 'aureo']

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4,probabilidade
5372,aureo,4,4,a,u,r,e,o,0.0


In [141]:
def conta_consoantes(palavra):
    
    contador = 0
    vogais = 'bcdfghjklmnpqrstvxywz'
    allowed = frozenset(vogais)
    
    for letra in palavra:
        if letra in vogais:
            contador += 1
    consoantes_unicas = len(allowed.intersection(palavra))
    
    return consoantes_unicas, contador


conta_consoantes('perde')   

(3, 3)

In [143]:
conta_consoantes('arroz')[0]   

2

In [144]:
df['n_consoantes'] = df.palavra.apply(lambda x: conta_consoantes(x)[1])
df['consoantes_unicas'] = df.palavra.apply(lambda x: conta_consoantes(x)[0])
df

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4,probabilidade,n_consoantes,consoantes_unicas
0,aarao,4,2,a,a,r,a,o,0.000000,1,1
1,abner,2,2,a,b,n,e,r,0.000000,3,3
2,acaia,4,2,a,c,a,i,a,0.000000,1,1
3,acker,2,2,a,c,k,e,r,0.000000,3,3
4,aires,3,3,a,i,r,e,s,0.000000,2,2
...,...,...,...,...,...,...,...,...,...,...,...
5419,unica,3,3,u,n,i,c,a,0.000761,2,2
5420,unico,3,3,u,n,i,c,o,0.000856,2,2
5421,urico,3,3,u,r,i,c,o,0.000000,2,2
5422,uteis,3,3,u,t,e,i,s,0.000190,2,2


In [145]:
consoantes_mais_usadas = 'srndm'

lista = []

for indice in consoantes_mais_usadas:
    lista.append(indice)



mask = df.letra_0.isin(lista) | df.letra_1.isin(lista) | df.letra_2.isin(lista) | df.letra_3.isin(lista) | df.letra_4.isin(lista)
df = df.loc[mask,:]
df


,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4,probabilidade,n_consoantes,consoantes_unicas
0,aarao,4,2,a,a,r,a,o,0.000000,1,1
1,abner,2,2,a,b,n,e,r,0.000000,3,3
3,acker,2,2,a,c,k,e,r,0.000000,3,3
4,aires,3,3,a,i,r,e,s,0.000000,2,2
5,alair,3,2,a,l,a,i,r,0.000000,2,2
...,...,...,...,...,...,...,...,...,...,...,...
5419,unica,3,3,u,n,i,c,a,0.000761,2,2
5420,unico,3,3,u,n,i,c,o,0.000856,2,2
5421,urico,3,3,u,r,i,c,o,0.000000,2,2
5422,uteis,3,3,u,t,e,i,s,0.000190,2,2


In [150]:
mask = (df.consoantes_unicas == 3) | (df.consoantes_unicas == 4)
df = df.loc[mask,:]
df.head()


,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4,probabilidade,n_consoantes,consoantes_unicas
1,abner,2,2,a,b,n,e,r,0.0,3,3
3,acker,2,2,a,c,k,e,r,0.0,3,3
9,alpes,2,2,a,l,p,e,s,0.0,3,3
11,andre,2,2,a,n,d,r,e,0.0,3,3
12,argel,2,2,a,r,g,e,l,0.0,3,3


# proximo passo seria pegar todas as palavras que mandou pro site e guardar tbm, pra tentar gerar um machine learning